In [ ]:
import sys
from pathlib import Path

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[1])

# Add to sys.path
if parent_dir not in sys.path:
  sys.path.insert(0, parent_dir)

from pprint import pprint
pprint(sys.path)

In [ ]:
import ffmpeg
import os

from pprint import pprint

In [ ]:
audio_file = '../../../tests/data/audio/0.wav'

In [ ]:
probe = ffmpeg.probe(audio_file)
audio = next((stream for stream in probe['streams'] if stream['codec_type'] == 'audio'), None)
print(f'Duration: {audio["duration"]}')